In [ ]:
from dask import dataframe as dd
import pandas as pd
import numpy as np

## Reading and preparing the data

In [ ]:
articles = dd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
transactions = dd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
customers = dd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
train, test = transactions.random_split([0.9, 0.1], random_state=43)
transactions["predictions"] = ''
submission = transactions[["customer_id","predictions"]].drop_duplicates(subset=["customer_id"]).compute()
print("Len of data: ", len(transactions))
del transactions

## Model

In [ ]:
most_sold_model = train.groupby(["article_id"]).agg({"article_id":"count"})\
                       .rename(columns={"article_id":"sold_amount"}).reset_index()\
                       .sort_values('sold_amount', ascending=False).head(12)

## Test

In [ ]:
top_k_test = test.groupby(["customer_id"])['article_id'].apply(lambda x: list(x), meta=("article_ids",object)).reset_index().compute().drop_duplicates(subset=["customer_id"])

In [ ]:
def top_k_score(predics, actual, k=12):
    scores = []
    actual = actual[:k]
    for i,pred in enumerate(predics):
        hit = actual.count(pred)
        if hit > 0:
            scores.append(hit / (i+1.0))
    
    if len(scores) > 0:
        return np.sum(scores) / len(set(actual))
    return 0

In [ ]:
top_k_test["score"] = top_k_test.apply(lambda x: top_k_score(most_sold_model.article_id.to_list(),x["article_ids"]),axis=1)
top_k_test.score.mean()

## Sample Submission

In [ ]:
submission["predictions"] = ' '.join(map(lambda x: str(x), most_sold_model.article_id.to_list()))
submission.head()